<a href="https://colab.research.google.com/github/rishiguptha54/Twitter_sentiment_analysis/blob/main/sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install kaggle

In [3]:
#configure the path of kaggle json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


## **Dependencies**

In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from nltk.stem import PorterStemmer

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## **Data Extraction**

In [8]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [9]:

twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


## **Data Preprocessing**

In [11]:
column_names=['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data.columns=column_names
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [13]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [14]:
twitter_data.replace({'target' :{4:1}}, inplace=True)

In [15]:
twitter_data['target'].value_counts()


,count
target,
0,800000
1,800000


## **Stemming**

In [16]:
port_stem = PorterStemmer()


In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [18]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [19]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [20]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [21]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [22]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [23]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [24]:
Y

array([0, 0, 0, ..., 1, 1, 1])

## **Testing & Training Data**

In [25]:
X_test,X_train,Y_test,Y_train=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

## **Vectorizing Data**

In [26]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [27]:
print(X_train)

  (0, 53904)	0.23662705066222536
  (0, 41330)	0.3621075009813407
  (0, 154815)	0.3501587145032445
  (0, 5650)	0.17295837060294847
  (0, 150603)	0.2214371376881977
  (0, 11837)	0.16338171192787918
  (0, 67046)	0.28383380524479984
  (0, 163496)	0.180932348330391
  (0, 26316)	0.2701195948137176
  (0, 158685)	0.3174206399662331
  (0, 107797)	0.17983918796121573
  (0, 51603)	0.25774294336856257
  (0, 104606)	0.4459147041194029
  (1, 136317)	0.5125311002730158
  (1, 56881)	0.48516752160641513
  (1, 83954)	0.35024490258566765
  (1, 134752)	0.4128221547743352
  (1, 141842)	0.21193862113578937
  (1, 99057)	0.2461083000957446
  (1, 2443)	0.2650067441796073
  (1, 5650)	0.181974929607951
  (2, 34981)	0.37310298419995075
  (2, 155497)	0.4431218247230756
  (2, 8454)	0.35890991973427855
  (2, 13273)	0.3887964554215523
  :	:
  (319994, 157662)	0.1692823231111289
  (319995, 114520)	0.4048122176535305
  (319995, 42067)	0.33609234404387445
  (319995, 173051)	0.3220764690826452
  (319995, 42660)	0.2995385

In [28]:
print(X_test)

  (0, 172332)	0.4425914290955845
  (0, 169740)	0.27246324011939344
  (0, 137247)	0.3578694854673394
  (0, 90675)	0.4219978835085155
  (0, 72344)	0.5336330797330684
  (0, 42634)	0.37277280423207615
  (1, 62896)	1.0
  (2, 172456)	0.3415940567458438
  (2, 168454)	0.32975845316340985
  (2, 158685)	0.15181999980261154
  (2, 158000)	0.18767650982964046
  (2, 157662)	0.3207869592151168
  (2, 111405)	0.16757521808028883
  (2, 102983)	0.24030158624165554
  (2, 69502)	0.16118699343993256
  (2, 58947)	0.1882195201728858
  (2, 51585)	0.20150931378777842
  (2, 50467)	0.2890070138255818
  (2, 48426)	0.1884626460390302
  (2, 42634)	0.4548661001770482
  (2, 30368)	0.31561094524694866
  (3, 159679)	0.27124577151803564
  (3, 157662)	0.2897977830373017
  (3, 150715)	0.3940397870474453
  (3, 67476)	0.373017355901726
  :	:
  (1279996, 160063)	0.23157245924834569
  (1279996, 151339)	0.27022281609718357
  (1279996, 144509)	0.43068224310653375
  (1279996, 123104)	0.2588982467230118
  (1279996, 112361)	0.22028

## **Logistic Regression**

In [29]:
model = LogisticRegression(max_iter=1000)

In [30]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

## **ACCURACY_SCORE**

In [31]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print("accuracy_score for training data:" , training_data_accuracy)

accuracy_score for training data: 0.815684375


In [32]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print("accuracy_score for test data :" , test_data_accuracy)

accuracy_score for test data : 0.77073671875


Model accuracy is 77%

## **Classification report**

In [33]:
print(classification_report(Y_test,X_test_prediction))


              precision    recall  f1-score   support

           0       0.78      0.75      0.77    640000
           1       0.76      0.79      0.78    640000

    accuracy                           0.77   1280000
   macro avg       0.77      0.77      0.77   1280000
weighted avg       0.77      0.77      0.77   1280000



## **Confusion Matrix**

In [34]:
print(confusion_matrix(Y_test,X_test_prediction))

[[480681 159319]
 [134138 505862]]


## **Saving Model**

In [35]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

## **Loading Saved Model**

In [37]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))